In [ ]:
import torch
import torchvision
#from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
from torchvision import models, transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import csv
import matplotlib.pyplot as plt
import numpy as np
import random
import os




######################## configure device
os.environ["CUDA_VISIBLE_DEVICES"]="0" # 1 is another GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


################## Set random seem for reproducibility
manualSeed = 9432
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
plt.ion()   # interactive mode
%matplotlib inline

The Steps for training a Neural Network are:

1. Load Data by creating a dataloader
2. Define the Network Model
3. Training the Network
4. Inference

1. Load Data : Dataloaders



In [ ]:
# Define the transformations to be used for Data Augmentagtion, 
apply_transform = transforms.Compose([transforms.Resize(32),transforms.ToTensor()])


In [ ]:
# Create Dataset

class MNIST_train(Dataset):
    # customized dataset
    def __init__(self, img_path, csv_name, transforms):
        
       
        
        img_nm=[]
        lbl=[]
        # read the entire csv file and save image_name, lbls
        with open(csv_name) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                img_nm.append(row[0])
                lbl.append(int(float(row[1])))
                
               
        
        self.img_nm=img_nm
        self.lbl=lbl
        self.img_path=img_path # dir of images.
        self.transform=transforms
        
      
        

        
        
    def __getitem__(self, index):
        ############# Return the sample data,gt for the input index
        # read the input image, gt
        # Read image as PIL
        tmp_img = Image.open(self.img_path+self.img_nm[index])
        # apply transform
        if self.transform is not None:
            tmp_img = self.transform(tmp_img)
        
        # convert label to tensor
        tmp_lbl=self.lbl[index]
        
        tmp_lbl=torch.from_numpy(np.array(tmp_lbl))
        tmp_lbl=tmp_lbl.long()
        
        
        
        return (tmp_img, tmp_lbl)
        

    def __len__(self):
        # Compute total number of samples in the dataset and return
        return len(self.lbl)

In [ ]:

train_dataset=MNIST_train(os.getcwd()+'/data/MNIST_cutsom_dataloader/Train/', os.getcwd()+'/data/MNIST_cutsom_dataloader/train_part.csv', apply_transform)

val_dataset=MNIST_train(os.getcwd()+'/data/MNIST_cutsom_dataloader/Train/', os.getcwd()+'/data/MNIST_cutsom_dataloader/val_part.csv', apply_transform)

test_dataset=MNIST_train(os.getcwd()+'/data/MNIST_cutsom_dataloader/Test/', os.getcwd()+'/data/MNIST_cutsom_dataloader/test.csv', apply_transform)

In [ ]:
(f, lbl) = train_dataset[0]
print(f.shape)
print(lbl)

fig=plt.figure(figsize=(12, 12)) 
fig.add_subplot(2, 1, 1) 
f=f.numpy()
f=np.squeeze(f)
plt.imshow(f, cmap='gray', vmin=0, vmax=1)

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32, 
                                           shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=32, shuffle=False, num_workers=2)


val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                          batch_size=32, shuffle=False, num_workers=2)



2. Defining the Network Model
![title](lenet1.png)


In [ ]:
# 1. Modified LeNet with Batch Norm Added

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Sequential(
                                    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, 
                                              stride=1, padding=0, dilation=1, groups=1, bias=False),
                                    nn.BatchNorm2d(6)
                                    )
        
        self.conv2 = nn.Sequential(
                                    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, 
                                              stride=1, padding=0, dilation=1, groups=1, bias=False),
                                    nn.BatchNorm2d(16)
                                    )
           
            
        self.fc1=nn.Sequential(
                                    nn.Linear( in_features=16*5*5, out_features=120, bias=False),
                                    nn.BatchNorm1d(120)
                                    )
        
        self.fc2=nn.Sequential(
                                    nn.Linear( in_features=120, out_features=84, bias=False),
                                    nn.BatchNorm1d(84)
                                    )
                                    
        
       
        self.fc3=nn.Linear( in_features=84, out_features=10, bias=True)
                                    

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        
        
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
    
    

In [ ]:
#2. Implement a CNN model of form: Conv+Maxpool->Conv+Maxpool with a variable "depth" number of blocks.
# The number of Convolution filters are doubled after each block. 
# The feature dimensionality at the end of the Conv layers must be fixed, independent of the input image size

### The Challenge is to write a code which generates the model in an iterative manner as depth is variable.
### Global Avg Pooling ensures that the output feature dim is independent of input image size
    # Other advantages: a) Low dimensionality of the FC layers   b) Explainability with CAM.


#This concept is often employed for constructing CNN.
#See for eg., DenseNet implementation: https://github.com/bamos/densenet.pytorch/blob/master/densenet.py line 99 onwards

class my_Conv(nn.Module):
    def __init__(self, inp_chnls, out_chnls):
        super(my_Conv, self).__init__()
        
        self.conv = nn.Sequential(
                                    nn.Conv2d(in_channels=inp_chnls, out_channels=out_chnls, kernel_size=3, 
                                              stride=1, padding=1, dilation=1, groups=1, bias=False),
                                    nn.BatchNorm2d(out_chnls),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                    )
       
    def forward(self, x):
        out = self.conv(x)
        return out
    
    
    
    
    
class my_CNN(nn.Module):
    def __init__(self, base_chnls, depth):
        super(my_CNN, self).__init__()
        
        layer_list=[]
        layer_list.append(my_Conv(1, base_chnls))
        for i in range(0, depth-1):
            layer_list.append(my_Conv(base_chnls, 2*base_chnls))
            base_chnls=base_chnls*2
            
        self.layers=nn.Sequential(*layer_list)
        
        self.fc=nn.Linear( in_features=base_chnls, out_features=10, bias=True)
       
    def forward(self, x):
        ftr = self.layers(x)
        ftr =  F.adaptive_avg_pool2d(ftr, (1,1))
        ftr = ftr.view(ftr.size(0), -1)
        out=self.fc(ftr)
        return out   

    
    



In [ ]:
###### Another way to code using ModuleList: see for example: 
# https://github.com/jvanvugt/pytorch-unet/blob/master/unet.py
# line 48-53 , 66-67

In [ ]:
#model=LeNet() # 1. Simple CNN: LeNet

model=my_CNN(base_chnls=2, depth=5)

model=model.to(device) # Transfer the model from CPU to GPU
print(model) # Print the model architecture



# Count no of learnable parameters in the model
def count_parameters(model):
    # Returns only trainable params due to the last if
    # wouldnot work for shared parameters which will be counted multiple times
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print( "No. of learnable Network Parameters= "+str(count_parameters(model)))

In [ ]:
def validate(val_loader, model, criterion):
    correct = 0 # Correctly predicted 
    total = 0 # Total number of samples
    running_loss=0
    
    model.eval()
    with torch.no_grad():
        for i, (images, labels) in enumerate(val_loader):
            
            images = images.to(device) # put data into gpu
            labels = labels.to(device)
                    
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
        
            # torch.max returns  a tuple (max_value, max_idx), the mx_idx gives the class label 
            predicted = torch.max(outputs, 1)[1] 
    
            # Compute Accuracy
            total += labels.size(0)
            correct += (predicted.detach() == labels).sum().item()

            running_loss=running_loss+loss.item() 
       
    
    
    
    
    acc=correct/total
    model.train()
    print ("\n Val_acc:  {:.4f}, Val_Classification Loss: {:.4f}"
                   .format(acc, running_loss/(i+1) ))
            
    return acc

In [ ]:
# Obtain an instance of loss and optimizer 
criterion = nn.CrossEntropyLoss() # This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class

init_lr=0.01
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

In [ ]:
############## TRAINING ###########
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
             
        

# Train the model
total_step = len(train_loader) # Number of batch updates in each epoch
curr_lr = init_lr
num_epochs=100 # maximum number of epochs for training

ptnc_cnt=0 # for Early stopping
patience=10 # Stop training if the val accuracy doesnot improve for "patience" epochs
max_metric=0 # best val accuracy encountered so far

for epoch in range(num_epochs):
    
    running_loss=0 # approximates the training loss by computing running average
    model.train() # Certain layers (for eg., BatchNorm and Dropout have different behaviours during training and inference)
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device) # put data into gpu
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        # Compute loss
        loss = criterion(outputs, labels)
        
        
        # Backpropagation
        optimizer.zero_grad() # # remove previous gradients
        loss.backward() # backpropagation through automatic gradient computation
        
        optimizer.step() # Update the optimizer parameters
        
        
        # Display Training Loss after every 25 batch of updates for current epoch
        running_loss=running_loss+loss.item()
        if (i+1) % 25 == 0:
            print ("Epoch [{}/{}], Batch [{}/{}] Train Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, running_loss/(i+1)), end ="\r")
            
            
    ######### End of An Epoch ##################
    
    # Decay learning rate
    if (epoch+1) % 50 == 0:
        curr_lr /= 10
        update_lr(optimizer, curr_lr)
        
    # Monitor validation Loss
    metric=validate(val_loader, model, criterion)
    
    # Checkpoint and Early Stopping 
    if metric>max_metric:
        nm='best_wt_CNN.pt'
        print("Val Performance improved, Saving checkpoint.. in "+nm)
        
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, nm)
        ptnc_cnt=0
        max_metric=metric
            
    else:
        ptnc_cnt=ptnc_cnt+1
        print('Validation metric has not improved in last '+str(ptnc_cnt)+' batch updates')
        if ptnc_cnt==patience:
            print("Early Stopping !")
            break
    

In [ ]:
##### Training is complete, now load the best training weight from the checkpoint
checkpoint = torch.load(nm)
model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
del checkpoint

metric=validate(test_loader, model, criterion)